In [15]:
#creating a wraper on top of wikipidia
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [16]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [30]:
api_wrapper=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=200)#this api is interacting with wikipedia to find the query results

wikipedia_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [18]:
!pip install arxiv



In [19]:
pip show arxiv


Name: arxiv
Version: 2.1.3
Summary: Python wrapper for the arXiv API: https://arxiv.org/help/api/
Home-page: https://github.com/lukasschwab/arxiv.py
Author: Lukas Schwab
Author-email: lukas.schwab@gmail.com
License: MIT
Location: c:\Users\Debadatta Nayak\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: feedparser, requests
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [31]:
wikipedia_tool

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Debadatta Nayak\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [32]:
wikipedia_tool.name

'wikipedia'

In [22]:
#taking a website and reading it as a data source by webbase loader

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader=WebBaseLoader("https://www.brightspeed.com/")
docs=loader.load()







In [23]:
docs

[Document(metadata={'source': 'https://www.brightspeed.com/', 'title': 'Brightspeed Internet | Fast and Reliable Fiber and Broadband Plans', 'description': 'Discover Brightspeeds ultrafast fiber and broadband internet plans with no bundles, annual contracts, or data caps. Check availability today!', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBrightspeed Internet | Fast and Reliable Fiber and Broadband Plans\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nBusiness\n\n\n\nCheck Availability\n                    \n\nAbout us\n\nContact\n\n\n\n\n\n\n\n\n\n\n\n\nShop internet\n\n\n\nStart streaming\n\n\n\nShop phone\n\n\n\nMoving?\n\n\n\nFind support\n                                \n\n\n\n                                                            \n                                                                Residential support\n                                                 

In [24]:
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

In [25]:
documents

[Document(metadata={'source': 'https://www.brightspeed.com/', 'title': 'Brightspeed Internet | Fast and Reliable Fiber and Broadband Plans', 'description': 'Discover Brightspeeds ultrafast fiber and broadband internet plans with no bundles, annual contracts, or data caps. Check availability today!', 'language': 'en'}, page_content='Brightspeed Internet | Fast and Reliable Fiber and Broadband Plans\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nBusiness\n\n\n\nCheck Availability\n                    \n\nAbout us\n\nContact\n\n\n\n\n\n\n\n\n\n\n\n\nShop internet\n\n\n\nStart streaming\n\n\n\nShop phone\n\n\n\nMoving?\n\n\n\nFind support\n                                \n\n\n\n                                                            \n                                                                Residential support\n                                                            \n                 

In [27]:
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings(openai_api_key=openai_api_key))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018E32BAFF10>)

In [28]:
from langchain.tools.retriever import create_retriever_tool

# Correctly assign the arguments by specifying the names
retrieval_tool = create_retriever_tool(
    name="brightspeed_search",  # Name of the tool
    retriever=retriever,        # Your retriever instance
    description="Search for information about brightspeed. For any question about brightspeed, you must use this tool!"
)


In [29]:
retrieval_tool

Tool(name='brightspeed_search', description='Search for information about brightspeed. For any question about brightspeed, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000018E241A1300>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018E32BAFF10>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000018E241A14E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018E32BAFF10>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

In [33]:
#combining all the tools

tools=[wikipedia_tool,retrieval_tool]

In [34]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Debadatta Nayak\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='brightspeed_search', description='Search for information about brightspeed. For any question about brightspeed, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000018E241A1300>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018E32BAFF10>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000018E241A14E0>, retriever=VectorStoreRetriever(tags=['F

In [37]:
from langchain.llms import OpenAI
llm=OpenAI(openai_api_key=openai_api_key,temperature=0.7,model='gpt-3.5-turbo')

C:\Users\Debadatta Nayak\AppData\Local\Temp\ipykernel_18328\560563363.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm=OpenAI(openai_api_key=openai_api_key,temperature=0.7,model='gpt-3.5-turbo')


In [40]:
pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


In [65]:
from langchain_core.prompts import ChatPromptTemplate

# Modify the prompt to include 'agent_scratchpad'
prompt = ChatPromptTemplate.from_template(
    '''Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.

Context:
{context}

Agent Scratchpad:
{agent_scratchpad}

Question:
{input}

Please provide your answer.'''
)


In [36]:
#querying from the specific tool by using Agents
#Agents:In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.


#here my agent will search for the query in the wikipedia_tool and if it's not finding anything then it'll go for retrieval_tool

In [87]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm=llm,prompt=prompt,tools=tools)

In [81]:
pip install --upgrade langchain


  Using cached langchain-0.2.16-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.2.38-py3-none-any.whl.metadata (6.2 kB)
Using cached langchain-0.2.16-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.2.38-py3-none-any.whl (396 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.36
    Uninstalling langchain-core-0.2.36:
      Successfully uninstalled langchain-core-0.2.36
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.15
    Uninstalling langchain-0.2.15:
      Successfully uninstalled langchain-0.2.15
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.14 requires langchain-core==0.2.36, but you have langchain-core 0.2.38 which is incompatible.


In [98]:
#Agent executer is needed to use the perticular agent

from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools)



In [99]:
agent_executor

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'context', 'input', 'intermediate_steps'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'context', 'input', 'intermediate_steps'], template='Answer the following question based only on the provided context.\nThink step by step before providing a detailed answer.\n\nContext:\n{context}\n\nIntermediate Steps:\n{intermediate_steps}\n\nAgent Scratchpad:\n{agent_scratchpad}\n\nQuestion:\n{input}\n\nPlease provide your answer.'))])
| RunnableBinding(bound=OpenAI(client=<openai.resources.completions.Completions object at 0x0000018E3528D850>, async_client=<openai.resources.completions.AsyncCompletions object at 0x0000018E35269950>, model_name='gpt-3.5-turbo', openai_api_key='sk-proj-OWP4NX5Q4GGi3bdopDPY2N7p

In [101]:
response = agent_executor.invoke(
    {
        "context": "Brightspeed is a company specializing in high-speed internet services.",
        "input": "Tell me about Brightspeed.",
        "agent_scratchpad": "",
        #"intermediate_steps": ""
    }
)


TypeError: Completions.create() got an unexpected keyword argument 'tools'

In [95]:
# Define the context, input, agent_scratchpad, and intermediate_steps
context = "Brightspeed is a company specializing in high-speed internet services and cutting-edge technology solutions."
input_question = "Tell me about brightspeed"
#agent_scratchpad = ""
#intermediate_steps = ""  # Provide appropriate values if needed

# Invoke the agent with all required variables
response = agent_executor.invoke({
    "context": context,
    "input": input_question,
   # "agent_scratchpad": agent_scratchpad,
    #"intermediate_steps": intermediate_steps
})

print(response)




> Entering new AgentExecutor chain...


TypeError: Completions.create() got an unexpected keyword argument 'tools'